In [3]:
import open3d as o3d
import numpy as np  



In [2]:
import cv2

# Load the image
image_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/src/diffusion_model/output/depth_map_inference/depth_bw/35e77eed59e1113c22e4620665c23c97_complete_pred.png"
image = cv2.imread(image_path)

# Define the cropping coordinates for each view (2 rows x 3 columns)
crop_coords = [
    (0, 0, 266, 400),    # View 1 (top-left)
    (266, 0, 532, 400),  # View 2 (top-middle)
    (532, 0, 800, 400),  # View 3 (top-right)
    (0, 400, 266, 800),  # View 4 (bottom-left)
    (266, 400, 532, 800),# View 5 (bottom-middle)
    (532, 400, 800, 800) # View 6 (bottom-right)
]

# Crop and save each view
for i, (x1, y1, x2, y2) in enumerate(crop_coords):
    cropped_image = image[y1:y2, x1:x2]  # Crop the image using numpy slicing
    cropped_path = f"cropped_view_{i+1}.png"
    cv2.imwrite(cropped_path, cropped_image)
    print(f"Saved cropped image {i+1} to {cropped_path}")


[ WARN:0@0.015] global loadsave.cpp:248 findDecoder imread_('/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/src/diffusion_model/output/depth_map_inference/depth_bw/35e77eed59e1113c22e4620665c23c97_complete_pred.png'): can't open/read file: check file path/integrity


TypeError: 'NoneType' object is not subscriptable

In [4]:
views = {
            "top": [0, 1, 0],
            "bottom": [0, -1, 0],
            "left": [1, 0, 0],
            "right": [-1, 0, 0],
            "front": [0, 0, -1],
            "back": [0, 0, 1]
        }

In [5]:
def set_camera_position(visualizer, center, distance, view_name="front"):
    ctr = visualizer.get_view_control()

    # Camera position will be distance units away from the center along the z-axis
    camera_position = center + np.array([0, 0, distance])

    # Set the camera parameters
    ctr.set_lookat(camera_position.tolist())  # Set the lookat point to the center of the scene
    ctr.set_front(views[view_name])  # Set the view direction)  # Camera looking straight down the z-axis
    
    # Adjust zoom if necessary
    if view_name in ["top", "bottom"]:
        ctr.set_up([0, 0, 1])
    else:
        ctr.set_up([0, 1, 0])


    # Apply the camera extrinsics
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    extrinsic = { 
        "left": np.array([
    [0, 0, 1, 0],    # Rotate to look along the positive x-axis
    [0, 1, 0, 0],    # No rotation around y
    [-1, 0, 0, distance],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "right": np.array([
    [0, 0, -1, 0],   # Rotate to look along the negative x-axis
    [0, 1, 0, 0],    # No rotation around y
    [1, 0, 0, distance],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "top": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, 1, 0],    # Rotate to look along the positive z-axis
    [0, -1, 0, distance],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "bottom": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, -1, 0],   # Rotate to look along the negative z-axis
    [0, 1, 0, distance],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "front": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, 1, distance],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ]),
        "back": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, -1, distance],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ])
    }
    
    camera_params.extrinsic = extrinsic[view_name]
    ctr.convert_from_pinhole_camera_parameters(camera_params, allow_arbitrary=True)
     # Set the lookat point to the center of the scene
    ctr.set_front(views[view_name]) # Camera looking straight down the z-axis
    if view_name in ["top", "bottom"]:
        ctr.set_up([0, 0, 1])
    else:
        ctr.set_up([0, 1, 0])

    visualizer.poll_events()
    visualizer.update_renderer()
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    return camera_params

In [6]:
window_width = 800
window_height = 800
vis = o3d.visualization.Visualizer()
vis.create_window(width=window_width, height=window_height)

mesh_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/data/raw/03001627/185bcb9bcec174c9492d9da2668ec34c/models/model_normalized.obj"
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()
#center the mesh
mesh.translate(-mesh.get_center())
vis.add_geometry(mesh)

fixed_distance = 1.0
camera_params = set_camera_position(vis, center=np.array([0, 0, 0]), distance=fixed_distance, view_name="front")

# Run the visualizer so you can manually adjust the viewpoint

ctr = vis.get_view_control()
vis.capture_depth_image("depth_image.png")



[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [7]:

#depth_image = o3d.io.read_image("depth_image.png")

depth_image = o3d.io.read_image("/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/notebooks/cropped_view_2.png")
pcd_from_depth = o3d.geometry.PointCloud.create_from_depth_image(
            depth_image,
            camera_params.intrinsic,
            camera_params.extrinsic,
        )

o3d.visualization.draw_geometries([pcd_from_depth, mesh])    

[Open3D WARNING] Read PNG failed: unable to parse header.


RuntimeError: [1;31m[Open3D Error] (static std::shared_ptr<PointCloud> open3d::geometry::PointCloud::CreateFromDepthImage(const Image &, const camera::PinholeCameraIntrinsic &, const Eigen::Matrix4d &, double, double, int, bool)) /Users/renes/development/open3d_work/Open3D/cpp/open3d/geometry/PointCloudFactory.cpp:151: Unsupported image format.
[0;m

: 